title: Launch of tier comparison in the upgrade flow    
author: Fabio Schmidt-Fischbach     
date: 2021-05-10      
region: EU    
link: https://docs.google.com/presentation/d/10DsZ6KyFQ73LlD7VDDocGpNEwjlpul9ZrvipFdRQIc4/edit?usp=sharing    
tags: memberships, premium, upgrade, upsell, funnel, tracking    
summary: We launched a tier comparison feature in the upgrade funnel that allows users to easily compare their current tier with the tier that they seek to upgrade to. Since this was not rolled out as a test, this deck analyses the core descriptive patterns we observed 

In [2]:
import pandas as pd
import altair as alt
import numpy as np
from causalimpact import CausalImpact

In [ ]:
query = """

select case when platform = 1 then 'android' 
			when platform = 2 then 'ios' 
			else 'web' 
		end as platform, 
		user_id, 
		collector_date,
		json_extract_path_text(se_label, 'step', True) as step 
from dbt.snowplow 
inner join dbt.zrh_users using (user_created)
where se_action = 'membership.upgrade-flow.view' 
	and collector_tstamp >= '2021-05-01' 
    and collector_tstamp < current_date

"""

In [ ]:
upgrades = """

select user_id, 
		dbt.zrh_user_product.product_id,
		subscription_valid_from
from dbt.zrh_user_product 
inner join dbt.zrh_users using (user_created)
where subscription_valid_from >= '2021-05-01' and enter_reason = 'UPGRADED'
    and subscription_valid_from < current_date

"""

In [ ]:
tier_comparison = """

select case when platform = 1 then 'android'
			when platform = 2 then 'ios' 
			else 'web' 
		end as platform, 
		collector_date,
		user_id 
from dbt.snowplow
inner join dbt.zrh_users using (user_created) 
where se_action = 'membership.upgrade-flow.click' 
and collector_tstamp >= '2021-05-01'
and json_extract_path_text(se_label, 'step', True) ilike  'compare%'
and collector_tstamp < current_date


"""

In [ ]:
card_selection = """

select case when platform = 1 then 'android' 
			when platform = 2 then 'ios' 
			else 'web' 
		end as platform, 
		user_id, 
		collector_date,
		json_extract_path_text(se_label, 'step', True) as step 
from dbt.snowplow 
inner join dbt.zrh_users using (user_created)
where se_action = 'membership.upgrade-flow.view' 
	and collector_tstamp >= '2021-05-01'
	and step = 'Card Selection'
    and collector_tstamp < current_date
    
"""

In [3]:
df = pd.read_csv("tier_comparison.csv")

df = df.groupby(["collector_date", "platform"])["user_id"].agg("nunique").reset_index()

alt.Chart(df).mark_line().encode(
    x="collector_date:O",
    y=alt.Y("user_id:Q", axis=alt.Axis(title="# of users who visit tier comparison")),
    color="platform:N",
).properties(width=500, height=500, title="# of users who look at tier comparison")

alt.Chart(...)

In [13]:
df = pd.read_csv("cta.csv")
df = df.loc[df["step"] == "cta", :]

df = df.groupby(["collector_date", "platform"])["user_id"].agg("nunique").reset_index()

tier = pd.read_csv("tier_comparison.csv").loc[
    :, ["user_id", "collector_date", "platform"]
]
tier = (
    tier.groupby(["collector_date", "platform"])["user_id"].agg("nunique").reset_index()
)

df = (
    df.merge(tier, on=["collector_date", "platform"], how="left")
    .reset_index()
    .fillna(0)
)

df["perc"] = df["user_id_y"] / df["user_id_x"]

alt.Chart(df.loc[df["collector_date"] >= "2021-05-01", :]).mark_line().encode(
    x="collector_date:O",
    y=alt.Y(
        "perc:Q",
        axis=alt.Axis(title="% of users that click on tier comparison", format="%"),
    ),
    color="platform:N",
).properties(width=500, height=500, title="% of users who look at tier comparison")

alt.Chart(...)

In [12]:
df = pd.read_csv("cta.csv")

df = df.loc[df["step"] == "cta", :]

df = df.groupby(["collector_date", "platform"])["user_id"].agg("nunique").reset_index()

df.index = pd.to_datetime(df["collector_date"])

df["smooth"] = df.groupby("platform")["user_id"].transform(
    lambda x: x.rolling(7, 1).mean()
)

alt.Chart(df.loc[df["collector_date"] >= "2021-01-01", :]).mark_line().encode(
    x="collector_date:O",
    y=alt.Y(
        "smooth:Q",
        axis=alt.Axis(title="# of users who enter the upgrade funnel (7d rolling avg)"),
    ),
    color="platform:N",
).properties(width=500, height=500, title="# of users who enter the upgrade funnel")

alt.Chart(...)

In [14]:
## cta --> bounce
## cta --> card selection
## cta --> compare plans --> bounce
## cta --> compare plans --> card selection

df = pd.read_csv("cta.csv")
df = df.loc[df["step"] == "cta", :]

df = df.groupby(["platform", "user_id", "collector_date"]).agg("count").reset_index()

compare = pd.read_csv("tier_comparison.csv")
compare = (
    compare.groupby(["platform", "user_id", "collector_date"])
    .agg("count")
    .reset_index()
)
compare["compare"] = 1

card = pd.read_csv("card_selection.csv")
card = (
    card.groupby(["platform", "user_id", "collector_date"]).agg("count").reset_index()
)
card["card"] = 1

df = (
    df.merge(compare, on=["user_id", "collector_date", "platform"], how="left")
    .reset_index()
    .fillna(0)
)
df = (
    df.merge(card, on=["user_id", "collector_date", "platform"], how="left")
    .reset_index()
    .fillna(0)
)

df = df.loc[:, ["user_id", "platform", "collector_date", "compare", "card"]]

df["group"] = "Bounced"
df.loc[(df["compare"] == 1) & (df["card"] == 0), "group"] = "Compare plans & bounce"
df.loc[(df["compare"] == 0) & (df["card"] == 1), "group"] = "Directly to card selection"
df.loc[
    (df["compare"] == 1) & (df["card"] == 1), "group"
] = "Compare plans & card selection"

df = (
    df.groupby(["collector_date", "platform", "group"])["user_id"]
    .agg("nunique")
    .reset_index()
)

df["perc"] = (
    100
    * df["user_id"]
    / df.groupby(["collector_date", "platform"])["user_id"].transform("sum")
)

alt.Chart(
    df.loc[
        (df["collector_date"] >= "2021-04-01")
        & (df["group"].isin(["Bounced", "Compare plans & bounce"]) == False)
        & (df["platform"] != "web"),
        :,
    ]
).mark_bar().encode(
    x="collector_date:O",
    y=alt.Y(
        "perc:Q", axis=alt.Axis(title="% of users at first screen in upgrade funnel")
    ),
    color="group:N",
    column="platform:N",
).properties(
    width=300, height=300, title="% of users at first screen in upgrade funnel"
)

alt.Chart(...)

In [15]:
## cta --> bounce
## cta --> card selection
## cta --> compare plans --> bounce
## cta --> compare plans --> card selection

df = pd.read_csv("cta.csv")
df = df.loc[df["step"] == "cta", :]

df = df.groupby(["platform", "user_id", "collector_date"]).agg("count").reset_index()

compare = pd.read_csv("tier_comparison.csv")
compare = (
    compare.groupby(["platform", "user_id", "collector_date"])
    .agg("count")
    .reset_index()
)
compare["compare"] = 1

card = pd.read_csv("card_selection.csv")
card = (
    card.groupby(["platform", "user_id", "collector_date"]).agg("count").reset_index()
)
card["card"] = 1

df = (
    df.merge(compare, on=["user_id", "collector_date", "platform"], how="left")
    .reset_index()
    .fillna(0)
)
df = (
    df.merge(card, on=["user_id", "collector_date", "platform"], how="left")
    .reset_index()
    .fillna(0)
)

df = df.loc[:, ["user_id", "platform", "collector_date", "compare", "card"]]

df["group"] = "Bounced"
df.loc[(df["compare"] == 1) & (df["card"] == 0), "group"] = "Compare plans & bounce"
df.loc[(df["compare"] == 0) & (df["card"] == 1), "group"] = "Directly to card selection"
df.loc[
    (df["compare"] == 1) & (df["card"] == 1), "group"
] = "Compare plans & card selection"

df = (
    df.groupby(["collector_date", "platform", "group"])["user_id"]
    .agg("nunique")
    .reset_index()
)

df["perc"] = (
    100
    * df["user_id"]
    / df.groupby(["collector_date", "platform"])["user_id"].transform("sum")
)

alt.Chart(
    df.loc[(df["collector_date"] >= "2021-04-01") & (df["platform"] != "web"), :]
).mark_bar().encode(
    x="collector_date:O",
    y=alt.Y(
        "perc:Q", axis=alt.Axis(title="% of users at first screen in upgrade funnel")
    ),
    color="group:N",
    column="platform:N",
).properties(
    width=300, height=300, title="% of users at first screen in upgrade funnel"
)

alt.Chart(...)

In [16]:
## cta --> bounce
## cta --> card selection
## cta --> compare plans --> bounce
## cta --> compare plans --> card selection

df = pd.read_csv("cta.csv")
df = df.loc[df["step"] == "cta", :]

df = df.groupby(["user_id", "collector_date"]).agg("count").reset_index()

compare = pd.read_csv("tier_comparison.csv")
compare = compare.groupby(["user_id", "collector_date"]).agg("count").reset_index()
compare["compare"] = 1

card = pd.read_csv("card_selection.csv")
card = card.groupby(["user_id", "collector_date"]).agg("count").reset_index()
card["card"] = 1

df = (
    df.merge(compare, on=["user_id", "collector_date"], how="left")
    .reset_index()
    .fillna(0)
)
df = (
    df.merge(card, on=["user_id", "collector_date"], how="left").reset_index().fillna(0)
)

df = df.loc[:, ["user_id", "collector_date", "compare", "card"]]

df["group"] = "Bounced"
df.loc[(df["compare"] == 1) & (df["card"] == 0), "group"] = "Compare plans & bounce"
df.loc[(df["compare"] == 0) & (df["card"] == 1), "group"] = "Directly to card selection"
df.loc[
    (df["compare"] == 1) & (df["card"] == 1), "group"
] = "Compare plans & card selection"

up = pd.read_csv("upgrades.csv")
up.columns = ["user_id", "product", "collector_date"]
up["collector_date"] = pd.to_datetime(up["collector_date"]).dt.date.astype(str)
up = up.groupby(["user_id"])["collector_date"].min().reset_index()
up["upgrade"] = 1

df = df.merge(up, on=["user_id", "collector_date"], how="left").fillna(0).reset_index()

df = df.groupby(["group"])["upgrade"].mean().reset_index()

alt.Chart(
    df.loc[df["group"].isin(["Bounced", "Compare plans & bounce"]) == False, :]
).mark_bar().encode(
    x="group:N",
    y=alt.Y("upgrade:Q", axis=alt.Axis(title="% that upgrade", format="%")),
    color="group:N",
).properties(
    width=400, height=300, title="% that upgrade"
)

alt.Chart(...)

In [ ]:
df = pd.read_csv("cta.csv")

df = df.loc[(df["platform"].isin(["ios", "android"])), :]

up = pd.read_csv("upgrades.csv")
up.columns = ["user_id", "product", "collector_date"]
up["collector_date"] = pd.to_datetime(up["collector_date"]).dt.date.astype(str)
up = up.groupby(["user_id"])["collector_date"].min().reset_index()
up["upgrade"] = 1

df = df.merge(up, on=["user_id", "collector_date"], how="left").reset_index().fillna(0)

df = (
    df.groupby(["collector_date"])["upgrade"]
    .agg(upgrades="sum", traffic="count")
    .reset_index()
)

df.index = pd.to_datetime(df.collector_date)

pre_period = ["2021-03-01", "2021-05-07"]
post_period = ["2021-05-08", "2021-05-13"]

df["weekday"] = pd.to_datetime(df["collector_date"]).dt.weekday
df["weekend"] = 1
df.loc[df["weekday"] <= 5, "weekend"] = 0

ci = CausalImpact(
    df.drop(columns=["collector_date", "weekday"]), pre_period, post_period
)

ci.plot()

In [ ]:
print(ci.summary("report"))

In [17]:
df = pd.read_csv("cta.csv")
df = df.loc[df["step"] == "cta", :]

up = pd.read_csv("upgrades.csv")
up.columns = ["user_id", "product", "collector_date"]
up["collector_date"] = pd.to_datetime(up["collector_date"]).dt.date.astype(str)
up = up.groupby(["user_id"])["collector_date"].min().reset_index()
up["upgrade"] = 1

df = df.merge(up, on=["user_id", "collector_date"], how="left").reset_index().fillna(0)

df = (
    df.groupby(["collector_date", "platform"])["upgrade"]
    .agg(upgrades="mean")
    .reset_index()
)

df.index = pd.to_datetime(df["collector_date"])

df["smooth"] = df.groupby("platform")["upgrades"].transform(
    lambda x: x.rolling(7, 1).mean()
)

alt.Chart(
    df.loc[(df["collector_date"] >= "2021-04-01") & (df["platform"] != "web"), :]
).mark_line().encode(
    x="collector_date:O",
    y=alt.Y("smooth:Q", axis=alt.Axis(title="% CR in upgrade funnel", format="%")),
    color="platform:N",
).properties(
    width=500, height=300, title="% CR in upgrade funnel (7d rolling average)"
)

alt.Chart(...)